This notebook explores both model index and opc ua scripts and contain examples of all the functions to make request to model index api and opc ua api servers.  

### Import Libraries

In [1]:
# Import the required packeages
import os
from dotenv import load_dotenv 
from pathlib import Path
import datetime

### Import Scripts

In [2]:
from pyprediktormapclient.opc_ua import OPC_UA
from pyprediktormapclient.model_index import ModelIndex
from pyprediktormapclient.auth_client import AUTH_CLIENT
from pyprediktormapclient.analytics_helper import AnalyticsHelper
from pyprediktormapclient.shared import *

In [ ]:
# Consider obtaining the envrionment variables from .env file if you are running this locally from source.
dotenv_path = Path("../.env")
load_dotenv(dotenv_path=dotenv_path)

In [4]:
username = os.environ["USERNAME"]
password = os.environ["PASSWORD"]
opcua_rest_url = os.environ["OPC_UA_REST_URL"]
opcua_server_url = os.environ["OPC_UA_SERVER_URL"]
model_index_url = os.environ["MODEL_INDEX_URL"]
ory_url = os.environ["ORY_URL"]

In [5]:
# Model index API
model = ModelIndex(url=model_index_url)

### Download data from modelindex api

In [ ]:
# Listed sites on the model index api server
namespaces = model.get_namespace_array()
namespaces

In [ ]:
# Types of Objects
object_types_json = model.get_object_types()
object_types = AnalyticsHelper(object_types_json)
object_types.dataframe

In [8]:
namespace_list = object_types.namespaces_as_list(namespaces)

# Initate the OPC UA API with a fixed namespace list
tsdata = OPC_UA(rest_url=opcua_rest_url, opcua_url=opcua_server_url, namespaces=namespace_list)

In [ ]:
# Unique types of Objects
object_types_unique = object_types.dataframe[["Id", "Name"]].drop_duplicates()
object_types_unique

In [ ]:
# To get typeId by type name of an object
object_type_id = model.get_object_type_id_from_name("SiteType")
object_type_id

In [ ]:
# To get the objects of a type
sites_json = model.get_objects_of_type("SiteType")

# Send the returned JSON into a normalizer to get Id, Type, Name, Props and Vars as columns
sites = AnalyticsHelper(sites_json)
sites.list_of_names()

In [ ]:
# Analytics helper
sites.variables_as_dataframe()

In [ ]:
sites.list_of_ids()

In [ ]:
# Selecting the second site
first_site_id = sites.list_of_ids()[1]
first_site_id

In [ ]:
# Get all stringsets for one park
string_sets_for_first_park_as_json = model.get_object_descendants(
    "StringSetType", [first_site_id], "PV_Assets"
)
string_sets_for_first_park = AnalyticsHelper(string_sets_for_first_park_as_json)
string_sets_for_first_park.dataframe

In [ ]:
# Ancestors of an object type, get all trackers that are ancestor of the parks string sets

trackers_as_json = model.get_object_ancestors(
    "TrackerType", string_sets_for_first_park.list_of_ids(), "PV_Serves"
)
trackers = AnalyticsHelper(trackers_as_json)
trackers.variables_as_dataframe()

### Download data from the opc ua api

In [ ]:
# Live value data of trackers
live_value = tsdata.get_values(
    trackers.variables_as_list(["AngleMeasured"])
)
live_value

In [ ]:
# Historic value data of trackers, 1 days worth of data 30 days ago
one_day_historic_tracker_data = tsdata.get_historical_aggregated_values(
    start_time=(datetime.datetime.now() - datetime.timedelta(30)),
    end_time=(datetime.datetime.now() - datetime.timedelta(29)),
    pro_interval=3600000,
    agg_name="Average",
    variable_list=trackers.variables_as_list(["AngleMeasured"]),
)
one_day_historic_tracker_data